In [99]:
from arcgis.gis import GIS, Item
import os
from multiprocessing import Process, Queue

from dotenv import load_dotenv, find_dotenv
load_dotenv(find_dotenv())

import sys
sys.path.append('../src')
sys.path.append('../')

import reach_tools
import hydrology
import config

gis = GIS(username=config.arcgis_username, password=config.arcgis_password)

url_reach_line = config.url_reach_line
url_reach_centroid = config.url_reach_centroid
url_reach_points = config.url_reach_points

lyr_line = reach_tools.ReachFeatureLayer(url_reach_line, gis)
lyr_centroid = reach_tools.ReachFeatureLayer(url_reach_centroid, gis)
lyr_points = reach_tools.ReachPointFeatureLayer(url_reach_points, gis)

reach_id = 2127

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [100]:
reach_id_fs = lyr_centroid.query(out_fields='reach_id', result_record_count=20, return_geometry=False)
reach_id_lst = [int(feat.attributes['reach_id']) for feat in reach_id_fs.features]
print(reach_id_lst[:20])

[1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20]


In [ ]:
for reach_id in reach_id_lst:
    
    reach = reach_tools.Reach.get_from_aw(reach_id)
    reach.snap_putin_and_takeout_and_trace()

    cent_resp = lyr_centroid.update_reach(reach)

    line_resp = lyr_line.update_reach(reach)

    putin_resp = lyr_points.query(f"reach_id = '{reach_id}' AND point_type = 'access' AND subtype = 'putin'", return_ids_only=True)['objectIds']
    if len(putin_resp):
        oid_putin = putin_resp[0]
        putin_feature = reach.putin.as_feature
        putin_feature.attributes['OBJECTID'] = oid_putin
        lyr_points.edit_features(updates=[putin_feature])
    else:
        lyr_points.edit_features(adds=[putin_feature])

    
    takeout_resp = lyr_points.query(f"reach_id = '{reach_id}' AND point_type = 'access' AND subtype = 'takeout'", return_ids_only=True)['objectIds']
    if len(takeout_resp):
        oid_takeout = takeout_resp[0]
        takeout_feature = reach.takeout.as_feature
        takeout_feature.attributes['OBJECTID'] = oid_takeout
        lyr_points.edit_features(updates=[takeout_feature])
    else:
        lyr_points.edit_features(adds=[takeout_feature])
        
    print(f'success: {reach.reach_id, reach.river_name, reach.reach_name}')

[13]
[9624]
success: ('1', 'Alsek', 'Haines Junction to Dry Bay')
